In [1]:
! python3 -m ipykernel install --user --name=main-venv

Installed kernelspec main-venv in /home/roman/.local/share/jupyter/kernels/main-venv


In [2]:
import os

import json

from tqdm import tqdm

import pandas as pd
import numpy as np

from pydantic import BaseModel, Field

from typing import List

from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from langchain_core.exceptions import OutputParserException

from langchain_gigachat.chat_models import GigaChat

from parser.static_parsers import build_hranidengi_parser
from parser.static_parsers import build_findozor_parser

In [3]:
DATA_DIR = "data"

# ✅Build Giga solution

In [4]:
class GigaOutput(BaseModel):

    topic: str = Field(description='''
        Название темы на форуме.
    ''')
    menace_status: str = Field(description='''
        Ответ от Гигачата на вопрос, обсуждается ли
        в этой теме нарушение закона с целью личной выгоды.
    '''),
    comment: str = Field(description='''
        Небольшой комментарий от Гигачата,
        почему он посчитал обсуждение в теме мошенническими.
    ''')
    highlights: str = Field(description='''
        Главные, по мнению гигачата, слова. 
    ''')

class GigaMenace:

    def __init__(self, token: str, prompt: str, model: str = 'GigaChat-2'):

        self.model = GigaChat(
            credentials=token,
            verify_ssl_certs=False,
            model=model
        ).with_structured_output(GigaOutput, method='json_mode')

        self.prompt = prompt
        self.messages = []

    def _init_chat(self, messages: List[str]):
        
        self.messages = [
            SystemMessage(content=self.prompt)
        ]

        for message in messages:
            self.messages.append(
                HumanMessage(content=str(message))
            )
    
    def inference(self, topics: List[str]):
        
        res = []
        
        for topic in tqdm(topics):

            try:
                self._init_chat(topics[topic])
                answer = self.model.invoke(
                    self.messages
                )
            except OutputParserException:
                answer = {'topic': topic, 'menace_status': 'да', 'comment': 'чувствительная тема', 'highlights': ''}
                
            res.append(dict(answer))
        
        return res

In [5]:
with open('gc-token.txt', 'r') as file:
    TOKEN = file.read()

system_prompt = '''
    Ты - помошник по борьбе с онлайн мошенничеством в банке.
    Тебе дают диалог на определенную тему на одном из онлайн банковских форумов.
    Твоя задача - определить, содержит ли диалог элементы мошенничества - 
    если да, ответь "да", иначе - "нет".
    Также дай небольшой комментарий, почему ты так решил.
    Главные слова в диалоге - подсвети.

    Проанализируй тему и верни ответ в формате JSON со структурой:
        {
          "topic": "оригинальное название темы"
          "menace_status": "да/нет",
          "comment": "твой короткомментарий",
          "highlights": "самые яркие тезисы диалога, если мошенничество, то явные слова о мошенничестве"
        }

    Запрещено:
        - Добавлять посторонний текст
        - Менять структуру JSON
'''

giga = GigaMenace(TOKEN, system_prompt)

# Hranidengi.com

In [6]:
hranidengi_parser = build_hranidengi_parser()

hranidengi_topics = pd.read_csv('data/hranidengi_mapping_credit_story.csv')
hranidengi_topics = hranidengi_topics[hranidengi_topics['menace_status'] == 'да']

hranidengi_topics_links = hranidengi_parser.parse_topics('https://hranidengi.com/forums/kreditnaja-istorija/')

In [7]:
hranidengi_topics_links

{'Незаконные запросы кредитной истории': 'https://hranidengi.com/threads/nezakonnye-zaprosy-kreditnoj-istorii.229/',
 'Обсуждение и анализ кредитной истории': 'https://hranidengi.com/threads/obsuzhdenie-i-analiz-kreditnoj-istorii.1119/',
 'Как бесплатно проверить кредитную историю': 'https://hranidengi.com/threads/kak-besplatno-proverit-kreditnuju-istoriju.401/',
 'Сервисы оплаты частями и их влияние на кредитную историю': 'https://hranidengi.com/threads/servisy-oplaty-chastjami-i-ix-vlijanie-na-kreditnuju-istoriju.669/',
 'Банк "Россия" завышает среднемесячные платежи': 'https://hranidengi.com/threads/bank-rossija-zavyshaet-srednemesjachnye-platezhi.920/',
 'Внесение изменений в кредитную историю': 'https://hranidengi.com/threads/vnesenie-izmenenij-v-kreditnuju-istoriju.231/',
 'Вопрос ламера: как испортить КИ любимому родственнику, за которым нужен присмотр, если нет ресурсов на круглосуточный присмотр.': 'https://hranidengi.com/threads/vopros-lamera-kak-isportit-ki-ljubimomu-rodstve

In [8]:
messages = dict()

for topic in hranidengi_topics['topic']:
    parsed = list(
        hranidengi_parser.parse_topic_comments(hranidengi_topics_links[topic], 5).values()
    )[0]
    parsed = ''.join(parsed).split('#')
    messages[topic] = parsed

In [9]:
hranidengi_res = giga.inference(messages)

100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.89s/it]


In [10]:
hranidengi_res

[{'topic': 'Вопрос ламера: как испортить КИ любимому родственнику, за которым нужен присмотр, если нет ресурсов на круглосуточный присмотр.',
  'menace_status': 'да',
  'comment': 'чувствительная тема',
  'highlights': ''}]

# Findozor.net

In [11]:
findozor_parser = build_findozor_parser()

findozor_topics = pd.read_csv('data/findozor_mapping_credit_story.csv')
# findozor_topics = findozor_topics[findozor_topics['menace_status'] == 'да']
findozor_topics = {
    'На авито предлагают за деньги разблокировать счёт в банке',
    'Рефинансирование в почта банке',
    'Фейковое место работы',
    'Кредиты',
    'Нудна помощь! Займов 6 штук, на 100 000, рефенанс, кредит что делать?',
    'Рефинансирование'
}

findozor_topics_links = findozor_parser.parse_topics('https://findozor.net/forum/forums/banki2/', deep = 5)

In [12]:
findozor_topics_links

{'Рефинансирование в почта банке': 'https://findozor.net/forum/threads/refinansirovaniye-v-pochta-banke.29579/',
 'На авито предлагают за деньги разблокировать счёт в банке': 'https://findozor.net/forum/threads/na-avito-predlagayut-za-den-gi-razblokirovat-schet-v-banke.25207/',
 'В каких банках одобряют карты на рассрочку': 'https://findozor.net/forum/threads/v-kakikh-bankakh-odobryayut-karty-na-rassrochku.29540/',
 'Вклад или накопительный счет — что посоветуете?': 'https://findozor.net/forum/threads/vklad-ili-nakopitel-nyi-schet-chto-posovetuyete.29433/',
 'Могут ли банки названивать часто, если не беру трубку?': 'https://findozor.net/forum/threads/mogut-li-banki-nazvanivat-chasto-yesli-ne-beru-trubku.26408/',
 'Всем привет 👋 где взять деньги с очень плохой кредитной историей?': 'https://findozor.net/forum/threads/vsem-privet-waving-hand-gde-vzyat-den-gi-s-ochen-plokhoi-kreditnoi-istoriyei.24028/',
 'Можно ли оформить кредитную карту при аресте в Сбер': 'https://findozor.net/forum/th

In [13]:
messages = dict()

for topic in findozor_topics:
    parsed = list(
        findozor_parser.parse_topic_comments(findozor_topics_links[topic], 5).values()
    )[0]
    parsed = ''.join(parsed).split('#')
    messages[topic] = parsed

In [14]:
findozor_res = giga.inference(messages)

100%|█████████████████████████████████████████████| 6/6 [00:06<00:00,  1.10s/it]


In [15]:
findozor_res

[{'topic': 'Заморозка суммы кредита для погашения',
  'menace_status': 'нет',
  'comment': 'Обсуждение носит информативный характер без явных признаков мошенничества.',
  'highlights': 'диалог посвящен вопросам реструктуризации долга и получения скидок от банков при наличии просрочек, без упоминания мошеннических схем'},
 {'topic': 'На авито предлагают за деньги разблокировать счёт в банке',
  'menace_status': 'да',
  'comment': 'чувствительная тема',
  'highlights': ''},
 {'topic': 'Фейковое место работы',
  'menace_status': 'да',
  'comment': 'чувствительная тема',
  'highlights': ''},
 {'topic': 'Рефинансирование в почта банке',
  'menace_status': 'да',
  'comment': 'чувствительная тема',
  'highlights': ''},
 {'topic': 'Рекомендации по закрытию займов в микрофинансовых организациях',
  'menace_status': 'нет',
  'comment': 'Обсуждение рекомендаций по управлению долгами в микрофинансовых организациях без признаков мошенничества.',
  'highlights': 'выходи на просрочки, обращайся в дру